In [1]:
!lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          16
On-line CPU(s) list:             0-15
Thread(s) per core:              2
Core(s) per socket:              8
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           85
Model name:                      Intel(R) Xeon(R) CPU @ 2.00GHz
Stepping:                        3
CPU MHz:                         1999.999
BogoMIPS:                        3999.99
Hypervisor vendor:               KVM
Virtualization type:             full
L1d cache:                       256 KiB
L1i cache:                       256 KiB
L2 cache:                        8 MiB
L3 cache:                        38.5 MiB
NUMA node0 CPU(s):               0-15
Vulnerabi

In [1]:
!pip install pydmd
!wget https://dl.dropboxusercontent.com/s/2g6poh9cikcykzd/cumulative-all.csv

  Using cached numpy-1.22.4-cp38-cp38-win_amd64.whl (14.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.4
    Uninstalling numpy-1.23.4:
      Successfully uninstalled numpy-1.23.4


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
whatstk 0.5.0 requires numpy~=1.19.1, but you have numpy 1.22.4 which is incompatible.
You should consider upgrading via the 'c:\users\aadha\anaconda3\python.exe -m pip install --upgrade pip' command.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
%%time

import pandas as pd
import numpy as np
from pydmd import DMD,HODMD

import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

import warnings
warnings.filterwarnings("ignore")

data=pd.read_csv(r'../Dataset/cumulative-all.csv') 
df = data.to_numpy()
countries = data.columns.tolist()

all_error=[]

for country in range(0,10):
  for error_thres in [0,1,5,10]:
    for train_days in [10,20,40,60]:
      for pred_days in [1,5,10,20]:

        prediction_window = pred_days
        training_window=train_days
        error_threshold=error_thres

        d_error = dict()
        full_predicted =[]

        for dd in range(1,training_window):

          iter=0

          predicted=[]
          groundtruth=[]


          train_country = df[:,country]
          train = train_country[0:training_window]
          total_length = train_country.shape[0]
          
            
          MAPE_error = 0
          def MAPE(y_true, y_pred): 
            y_true, y_pred = np.array(y_true), np.array(y_pred)
            return np.mean(np.abs((y_true - y_pred) / np.maximum(np.ones(len(y_true)), np.abs(y_true))))*100

          #print(f"\nStarting training with {total_length} days from {iter} - {iter+training_window} days \n")
          while iter+training_window<=train_country.shape[0] - prediction_window:

            if MAPE_error>error_threshold:
              train = train_country[iter:iter+training_window]
              #print(f"\n.........Updating training data {iter} - {iter+training_window} days\n")
            
            try:
              dmd = HODMD(svd_rank=0, tlsq_rank=0, exact=True, opt=True, rescale_mode='auto', d=dd) # Specify desired truncation   
              dmd.fit(train) # Fit the model on the first 5 rows
              dmd.dmd_time['tend'] += prediction_window # Predict one additional time step
              recon = dmd.reconstructed_data.real.T # Make prediction
              
              #print("Predicting for Days: ", iter+training_window, iter+training_window+(prediction_window))

              predicted.append(recon.squeeze()[-prediction_window:])
              groundtruth.append(train_country[iter+training_window:iter+training_window+prediction_window])

              MAPE_error = MAPE(recon.squeeze()[-prediction_window:], train_country[iter+training_window:iter+training_window+prediction_window])
              iter +=prediction_window
            except: break

          alldaysMAPE = MAPE(np.array(groundtruth).flatten(),np.array(predicted).flatten())
          alldaysMAPE = np.nan_to_num(alldaysMAPE, nan=1000, posinf=100000.0, neginf=10000.0)
          d_error[alldaysMAPE] = np.array(predicted).flatten()

        print(f"Error{ round(np.array(list(d_error.keys())).min(), 3)}%")
        all_error.append((countries[country], training_window, prediction_window, error_threshold, round(np.array(list(d_error.keys())).min(), 3)))
        plt.figure()
        plt.plot(train_country[training_window:], label='True')
        plt.plot(d_error[np.array(list(d_error.keys())).min()], label='Predicted')
        plt.xlabel("Number of days")
        plt.ylabel("Number of infected cases")
        plt.ylim(0,train_country.max())
        plt.legend()
        plt.title(f"{countries[country]} - Training {training_window} days Predicting {prediction_window} days with Error threshold {error_threshold}", size=8)

        plt.savefig(f"{countries[country]} - Training {training_window} days Predicting {prediction_window} days with Error threshold {error_threshold}.png", dpi=600)
        plt.show()
        plt.close()

FileNotFoundError: [Errno 2] No such file or directory: '../Dataset/cumulative-all.csv'

In [2]:
df = pd.DataFrame(all_error)
df.to_csv("Adaptive_HODMD new results_all_countries.csv")

In [5]:
np.version


<module 'numpy.version' from 'C:\\Users\\aadha\\anaconda3\\lib\\site-packages\\numpy\\version.py'>

In [3]:
!pwd

/home/jovyan


In [1]:
import shutil,os,glob

In [ ]:
shutil.make_archive("HODMD", 'zip', '/home/jovyan')

In [2]:
os.makedirs("HODMD_results")

In [6]:
glob.glob("/home/jovyan/*.png")

['/home/jovyan/Germany - Training 10 days Predicting 1 days with Error theshold 0.png',
 '/home/jovyan/Germany - Training 10 days Predicting 5 days with Error theshold 0.png',
 '/home/jovyan/Germany - Training 10 days Predicting 10 days with Error theshold 0.png',
 '/home/jovyan/Germany - Training 10 days Predicting 20 days with Error theshold 0.png',
 '/home/jovyan/Germany - Training 20 days Predicting 1 days with Error theshold 0.png',
 '/home/jovyan/Germany - Training 20 days Predicting 5 days with Error theshold 0.png',
 '/home/jovyan/Germany - Training 20 days Predicting 10 days with Error theshold 0.png',
 '/home/jovyan/Germany - Training 20 days Predicting 20 days with Error theshold 0.png',
 '/home/jovyan/Germany - Training 40 days Predicting 1 days with Error theshold 0.png',
 '/home/jovyan/Germany - Training 40 days Predicting 5 days with Error theshold 0.png',
 '/home/jovyan/Germany - Training 40 days Predicting 10 days with Error theshold 0.png',
 '/home/jovyan/Germany - Tr

In [7]:
for i in glob.glob("/home/jovyan/*.png"):
    shutil.move(i, "HODMD_results/")